In [1]:
import copy
import qcodes
#from qcodes import StandardParameter

In [2]:
class Powersource(qcodes.Instrument):
    def __init__(self, name): 
        super().__init__(name)
        self._voltage = 0
        
        self.add_parameter(
            name="voltage", 
            set_cmd=self._set_voltage,
            get_cmd=self._get_voltage,
            unit="mV"  # Stupid thing works in mV instead of Volt
        )
        
    def _set_voltage(self, value): 
        self._voltage = value
        
    def _get_voltage(self):
        return self._voltage

In [3]:
# This cell works, but maybe its not how we want to do things in the long run 

class LinearMapping: 
    def __init__(self, conversion_factor): 
        self._conversion_factor = conversion_factor
    
    def __call__(self, value): 
        return value * self._conversion_factor
    
    def inverse(self, value): 
        return value / self._conversion_factor

def alias(parameter, name, mapping, **kwargs):
    
    name = name or parameter.name
    
    if isinstance(mapping, float) or isinstance(mapping, int): 
        mapping = LinearMapping(mapping)
    
    instrument = parameter._instrument  # We are accessing a provate variable here, which is not ideal 
    
    instrument.add_parameter(
        name=name, 
        set_cmd=lambda value: parameter(mapping(value)), 
        get_cmd=lambda: mapping.inverse(parameter()), 
        **kwargs
    )
    
    return getattr(instrument, name)

In [11]:
# This cell does not work (yet), but maybe this is the way forward 

def alias(parameter, mapping, **kwargs): 
    
    if isinstance(mapping, float) or isinstance(mapping, int): 
        mapping = LinearMapping(mapping)
    
    # The proposal is to add a copy method on the Parameter base class. The copy method shall return a new parameter 
    # of the same type and with the same attributes (including instrument) as the original, except where this is overriden 
    # by the copy arguments. 
    new_parameter = parameter.copy(
        set_cmd=lambda value: parameter(mapping(value)), 
        get_cmd=lambda: mapping.inverse(parameter()), 
        **kwargs
    )
    
    return new_parameter

In [4]:
pwr = Powersource("pwr")

In [5]:
gate = alias(pwr.voltage, name="gate", mapping=1E-3, unit="V", label="gate voltage")

In [6]:
gate(3) # Set at 3 V

In [7]:
gate()

3.0

In [8]:
pwr.voltage()

0.003

In [9]:
pwr.snapshot()

{'__class__': '__main__.Powersource',
 'functions': {},
 'name': 'pwr',
 'parameters': {'IDN': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'IDN',
   'name': 'IDN',
   'ts': None,
   'unit': '',
   'vals': '<Anything>',
   'value': None},
  'gate': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'gate voltage',
   'name': 'gate',
   'ts': '2017-11-16 14:36:53',
   'unit': 'V',
   'vals': '<Numbers>',
   'value': 3.0},
  'voltage': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'voltage',
   'name': 'voltage',
   'ts': '2017-11-16 14:36:55',
   'unit': 'mV',
   'vals': '<Numbers>',
   'value': 0.003}},
 'submodules': {}}

In [10]:
pwr.voltage.snapshot()

{'__class__': 'qcodes.instrument.parameter.StandardParameter',
 'instrument': '__main__.Powersource',
 'instrument_name': 'pwr',
 'label': 'voltage',
 'name': 'voltage',
 'ts': '2017-11-16 14:36:55',
 'unit': 'mV',
 'vals': '<Numbers>',
 'value': 0.003}

In [11]:
gate.snapshot()

{'__class__': 'qcodes.instrument.parameter.StandardParameter',
 'instrument': '__main__.Powersource',
 'instrument_name': 'pwr',
 'label': 'gate voltage',
 'name': 'gate',
 'ts': '2017-11-16 14:36:53',
 'unit': 'V',
 'vals': '<Numbers>',
 'value': 3.0}

In [12]:
gate.metadata

{}

In [13]:
pwr.voltage.metadata

{}